<a href="https://colab.research.google.com/github/mayrisnawati/Capstone-Project/blob/master/Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set the seed
import numpy as np
np.random.seed(40)
tf.random.set_seed(40)

In [ ]:
TARGET_SIZE = (150,150)
BATCH_SIZE = 32
TRAINING_DIR = "/content/drive/MyDrive/FIX DATA/training"
VALIDATION_DIR = "/content/drive/MyDrive/FIX DATA/validation"

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      target_size=TARGET_SIZE,
                                                      class_mode='categorical',
                                                      batch_size=BATCH_SIZE,
                                                      shuffle=False
                                                      )
  
validation_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                target_size=TARGET_SIZE,
                                                                class_mode='categorical',
                                                                batch_size=BATCH_SIZE,
                                                                shuffle=False
                                                                )  

Found 2844 images belonging to 2 classes.
Found 317 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
        # This is the first convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # This is the second convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

         # Add the same layers as before
        tf.keras.layers.Flatten(),
        
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='sigmoid')  
])

model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=20,
                    verbose=2,
                    validation_data=validation_generator,
                   )


Epoch 1/20
89/89 - 58s - loss: 1.0064 - accuracy: 0.5977 - val_loss: 0.5763 - val_accuracy: 0.5836 - 58s/epoch - 647ms/step
Epoch 2/20
89/89 - 51s - loss: 0.6412 - accuracy: 0.6832 - val_loss: 0.5158 - val_accuracy: 0.7508 - 51s/epoch - 569ms/step
Epoch 3/20
89/89 - 49s - loss: 0.5819 - accuracy: 0.7493 - val_loss: 0.5416 - val_accuracy: 0.7066 - 49s/epoch - 555ms/step
Epoch 4/20
89/89 - 50s - loss: 0.6514 - accuracy: 0.7423 - val_loss: 0.6537 - val_accuracy: 0.6246 - 50s/epoch - 564ms/step
Epoch 5/20
89/89 - 50s - loss: 0.4857 - accuracy: 0.7957 - val_loss: 0.7392 - val_accuracy: 0.6814 - 50s/epoch - 565ms/step
Epoch 6/20
89/89 - 49s - loss: 0.4663 - accuracy: 0.7989 - val_loss: 0.6673 - val_accuracy: 0.7035 - 49s/epoch - 554ms/step
Epoch 7/20
89/89 - 51s - loss: 0.6488 - accuracy: 0.7897 - val_loss: 0.5403 - val_accuracy: 0.7003 - 51s/epoch - 569ms/step
Epoch 8/20
89/89 - 55s - loss: 0.4993 - accuracy: 0.7838 - val_loss: 0.4265 - val_accuracy: 0.8076 - 55s/epoch - 622ms/step
Epoch 9/

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
from tensorflow.keras import Model


pretrained_model = InceptionV3(input_shape=(150,150,3),
                                include_top=False,
                                weights='imagenet')

inputs2 = pretrained_model.input

x = tf.keras.layers.Flatten()(pretrained_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs2 = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs2, outputs=outputs2)

model.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                             

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.9:                 
                print("\nReached 90% accuracy so cancelling training!")
                
                # Stop training once the above condition is met
                self.model.stop_training = True

In [ ]:
# Instantiate the callback class
callbacks = myCallback()
# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=20,
                    verbose=2,
                    validation_data=validation_generator,
                    callbacks=[callbacks])



Epoch 1/20
89/89 - 56s - loss: 0.7140 - accuracy: 0.5000 - val_loss: 0.7292 - val_accuracy: 0.5000 - 56s/epoch - 625ms/step
Epoch 2/20
89/89 - 56s - loss: 0.7111 - accuracy: 0.5000 - val_loss: 118.4529 - val_accuracy: 0.5000 - 56s/epoch - 630ms/step
Epoch 3/20
89/89 - 54s - loss: 0.7124 - accuracy: 0.5000 - val_loss: 6421.4775 - val_accuracy: 0.5000 - 54s/epoch - 603ms/step
Epoch 4/20
89/89 - 53s - loss: 0.7129 - accuracy: 0.5000 - val_loss: 0.7497 - val_accuracy: 0.5000 - 53s/epoch - 598ms/step
Epoch 5/20
89/89 - 54s - loss: 0.7081 - accuracy: 0.5000 - val_loss: 0.6960 - val_accuracy: 0.5000 - 54s/epoch - 610ms/step
Epoch 6/20
89/89 - 53s - loss: 0.7100 - accuracy: 0.5000 - val_loss: 1.6988 - val_accuracy: 0.5000 - 53s/epoch - 594ms/step
Epoch 7/20
89/89 - 53s - loss: 0.7057 - accuracy: 0.5000 - val_loss: 0.8666 - val_accuracy: 0.5000 - 53s/epoch - 596ms/step
Epoch 8/20
89/89 - 52s - loss: 0.7037 - accuracy: 0.5000 - val_loss: 0.9727 - val_accuracy: 0.5000 - 52s/epoch - 587ms/step
Epo

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

class_names = ['crack', 'pothole']

for fn in uploaded.keys():
 
  # Memprediksi gambar
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predictions = model.predict(images, batch_size=10)
  predicted_class_index = np.argmax(predictions[0])
  predicted_class = class_names[predicted_class_index]

  print(f"{fn} is {predicted_class}")

Saving south-korea-road-e1472793947125.jpg to south-korea-road-e1472793947125.jpg
1/1 [==============================] - 0s 18ms/step
south-korea-road-e1472793947125.jpg is crack


In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

class_names = ['crack', 'pothole']

for fn in uploaded.keys():
 
  # Memprediksi gambar
  path = '/content/' + fn
  img = load_img(path, target_size=(28, 28))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predictions = model.predict(images, batch_size=10)
  predicted_class_index = np.argmax(predictions[0])
  predicted_class = class_names[predicted_class_index]

  print(f"{fn} is {predicted_class}")

Saving CFD_002.jpg to CFD_002 (1).jpg


InvalidArgumentError: ignored